# Test a meta-reinforcement learning model (A2C) on the two-step task

## What is meta-reinforcement learning?

TODO: explain meta-reinforcement learning

### Let's setup the environment

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## What is the two step task, and why should I care?

P1: explain model-free and model-based behavior in cognitive science

P2: two-step task is an iconic experiment demonstrating that humans have the ability to learn in a model-based manner.

P3: add maybe a figure to explain how the two-step task should work.

## Finally, let's train the model

In [ ]:
from envs.GuessBoundary import GuessBoundaryTask
from utils.utils import do_experiment

# specify the agent and the environment
num_lstm_units = 10
num_train_steps = 20000000
policy_kwargs = {'n_lstm': num_lstm_units, 'action_noise': 0}
env = GuessBoundaryTask(obs_mode='all_history')

do_experiment(env, num_train_steps, policy_kwargs)

## Let's visualize the model performance

In [2]:
from stable_baselines.common.cmd_util import make_vec_env

from utils.utils import Simulation
from envs.GuessBoundary import GuessBoundaryTask
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

sims = [Simulation('./outputs/GuessBoundary/A2C-LSTM10')]
sims[0].evaluate(num_test_episodes=100, num_trials=50, verbose=True)

/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/

invalid load_params_file_idx. Loading parameters from the latest save file.
average reward per episode is  1.0
average episode length is  9.74


In [ ]:
from stable_baselines.common.cmd_util import make_vec_env

from utils.utils import Simulation
from utils.visualization import plot_best_action_reversal, calc_stay_prob, plot_stay_prob, visualize_hidden_units
from envs.TwoStepTask import TwoStepTask
from envs.GuessBoundary import GuessBoundaryTask

env = TwoStepTask(prob_switch_reward=0.05, num_trials=200)
env = make_vec_env(lambda: env, n_envs=1)

sims = [Simulation('./outputs/A2C-LSTM10_Standard_PSwitch5e-2/train25000', env = env),
              Simulation('./outputs/A2C-LSTM10_Standard_PSwitch5e-2/train500000', env = env)]
            
fig, axs = plt.subplots(3, len(sims), figsize = (16, 16))
fig.suptitle('Training Steps: 25K / 500K', fontsize=30)
for ctr, sim in enumerate(sims):
    sim.evaluate()
    
    # first row: how fast can the model switch behaviors after a switch
    plotActionReversal(axs[0, ctr], sim.TestRollouts)
    
    # second row: MB or MF behavior?
    stayProb, _, _ = calcStayProb(sim.TestRollouts)
    plotStayProb(axs[1, ctr], stayProb)
    
    # third row: try to visualize the hidden units
    vis = visualizeH(axs[2, ctr], sim.TestRollouts)